In [2]:
import pandas as pd
import dask.dataframe as dd
import numpy as np

In [3]:
pandas_df = pd.read_pickle("./raw_weekly_df.pkl")
dask_df = dd.from_pandas(pandas_df, npartitions=8)
mean = 0
std = 0

In [3]:
print(pandas_df.head())

   dept_id cat_id      item_id state_id store_id   datetime  sales
0  FOODS_1  FOODS  FOODS_1_001       CA     CA_1 2011-01-31    3.0
1  FOODS_1  FOODS  FOODS_1_001       CA     CA_1 2011-02-07    9.0
2  FOODS_1  FOODS  FOODS_1_001       CA     CA_1 2011-02-14    9.0
3  FOODS_1  FOODS  FOODS_1_001       CA     CA_1 2011-02-21    8.0
4  FOODS_1  FOODS  FOODS_1_001       CA     CA_1 2011-02-28   14.0


In [4]:
def standardize(input_ser):
    input_std = np.std(input_ser)
    input_mean = input_ser.mean()
    global mean
    global std
    mean = input_mean
    std = input_std
    
    return (input_ser-input_mean)/input_std

In [5]:
#Unstandarizes a series, given it's mean and std.
def unstandardize(ser):
    return ser*std + mean

In [6]:
print(pandas_df['sales'].head())

0     3.0
1     9.0
2     9.0
3     8.0
4    14.0
Name: sales, dtype: float32


In [7]:
b = standardize(pandas_df['sales'])
print(b)
print(mean)
print(std)

0         -0.259046
1         -0.020909
2         -0.020909
3         -0.060599
4          0.177539
             ...   
6841116   -0.378115
6841117   -0.378115
6841118   -0.378115
6841119   -0.378115
6841120   -0.378115
Name: sales, Length: 6841121, dtype: float32
9.526814460754395
25.195552825927734


In [8]:
print(unstandardize(b).head())

0     3.0
1     9.0
2     9.0
3     8.0
4    14.0
Name: sales, dtype: float32


In [9]:
def stat_feature(df,grouping,col):
    #TODO copying dataframes doesn't scale well
    
    
    #TODO instead of passing mean, min, and max, calculate them all at once using a single line of code and an \
    #  aggregate_dict param (hint: use .agg())
    #TODO if we include the current row's data in an aggregate calculation, we'll "leak" data about the future when \
    # making predictions. shift your dataframe back one period  to avoid this issue
    agg = df.groupby([grouping])[col].agg(['mean','min','max'])
    cumsum = df.groupby([grouping])[col].shift(periods = 1).agg(['cumsum'])
    return agg,cumsum 

In [10]:
agg,cumsum = stat_feature(pandas_df,'state_id','sales')
print(agg)
print(cumsum)

               mean  min          max
state_id                             
CA        10.585984  0.0  4220.000488
TX         9.033415  0.0  1871.000122
WI         8.882002  0.0  1755.000122
             cumsum
0               NaN
1               3.0
2              12.0
3              21.0
4              29.0
...             ...
6841116  65174088.0
6841117  65174088.0
6841118  65174088.0
6841119  65174088.0
6841120  65174088.0

[6841121 rows x 1 columns]


In [11]:
def detrend_feature(df,grouping,col):
    return df[col] - df[col].shift(periods=1)

In [12]:
print(detrend_feature(pandas_df,'state_id','sales'))

0          NaN
1          6.0
2          0.0
3         -1.0
4          6.0
          ... 
6841116    0.0
6841117    0.0
6841118    0.0
6841119    0.0
6841120    0.0
Name: sales, Length: 6841121, dtype: float32


In [36]:
def month_feature(dataframeinput):
    return dataframeinput['datetime'].dt.month

In [37]:
month_feature(pandas_df)

0          1
1          2
2          2
3          2
4          2
          ..
6841116    5
6841117    5
6841118    6
6841119    6
6841120    6
Name: datetime, Length: 6841121, dtype: int64

In [10]:
def quarter_feature(dataframeinput):
    return dataframeinput['datetime'].dt.quarter
    #January-March = 1, April-June = 2, etc.

In [11]:
quarter_feature(pandas_df)

0          1
1          1
2          1
3          1
4          1
          ..
6841116    2
6841117    2
6841118    2
6841119    2
6841120    2
Name: datetime, Length: 6841121, dtype: int64

In [12]:
def week_of_year_feature(dataframeinput):
    return dataframeinput['datetime'].dt.weekofyear

In [13]:
week_of_year_feature(pandas_df)

0           5
1           6
2           7
3           8
4           9
           ..
6841116    21
6841117    22
6841118    23
6841119    24
6841120    25
Name: datetime, Length: 6841121, dtype: int64

In [14]:
def day_of_week_feature(dataframeinput):
    return dataframeinput['datetime'].dt.dayofweek

In [15]:
day_of_week_feature(pandas_df)

0          0
1          0
2          0
3          0
4          0
          ..
6841116    0
6841117    0
6841118    0
6841119    0
6841120    0
Name: datetime, Length: 6841121, dtype: int64

In [38]:
def month_year_feature(dataframeinput):
    year = dataframeinput['datetime'].dt.year
    month = month_feature(dataframeinput)
    newdataframe = pd.DataFrame()
    newdataframe['month'] = month
    newdataframe['year'] = year
    return(newdataframe)

In [39]:
month_year_feature(pandas_df)

,month,year
0,1,2011
1,2,2011
2,2,2011
3,2,2011
4,2,2011
...,...,...
6841116,5,2016
6841117,5,2016
6841118,6,2016
6841119,6,2016


In [28]:
def semester_feature(dataframeinput):
    dataframeinputcopy = dataframeinput
    dataframeinputcopy['quarter'] = quarter_feature(dataframeinput)
    dataframeinputcopy['semester'] = np.where(dataframeinputcopy.quarter.isin([1,2]),1,2)
    return dataframeinputcopy[['datetime', 'semester']].head()

In [29]:
semester_feature(pandas_df)

,datetime,semester
0,2011-01-31,1
1,2011-02-07,1
2,2011-02-14,1
3,2011-02-21,1
4,2011-02-28,1
